In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

In [4]:
Data = pd.read_excel('/content/BASE_FINAL.xlsx')

In [5]:
Data.head()

,Unnamed: 0,EDAD,ESTU_GENERO,ESTU_DEPTO_RESIDE,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,FAMI_TIENEINTERNET,FAMI_TIENECOMPUTADOR,ESTU_DEDICACIONLECTURADIARIA,ESTU_HORASSEMANATRABAJA,COLE_BILINGUE,FAMI_COMECARNEPESCADOHUEVO,COLE_NATURALEZA,PUNT_GLOBAL
0,71,18,F,BOGOTA,Educacion profesional completa,Educacion profesional completa,Si,Si,Entre 1 y 2 horas,0,S,Todos o casi todos los dias,NO OFICIAL,338
1,157,30,F,BOGOTA,Ninguno,Secundaria (Bachillerato) completa,Si,Si,30 minutos o menos,0,N,1 o 2 veces por semana,NO OFICIAL,235
2,159,19,M,BOGOTA,No sabe,Secundaria (Bachillerato) incompleta,Si,Si,No leo por entretenimiento,Entre 11 y 20 horas,N,Todos o casi todos los dias,NO OFICIAL,268
3,161,11,F,CUNDINAMARCA,Ninguno,Primaria completa,Si,Si,30 minutos o menos,mas de 30 horas,N,Todos o casi todos los dias,NO OFICIAL,188
4,188,18,F,BOGOTA,Secundaria (Bachillerato) incompleta,Educacion profesional incompleta,Si,Si,Entre 30 y 60 minutos,0,N,Todos o casi todos los dias,NO OFICIAL,320


In [6]:
Nombres = Data.columns ; Nombres

Index(['Unnamed: 0', 'EDAD', 'ESTU_GENERO', 'ESTU_DEPTO_RESIDE',
       'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'FAMI_TIENEINTERNET',
       'FAMI_TIENECOMPUTADOR', 'ESTU_DEDICACIONLECTURADIARIA',
       'ESTU_HORASSEMANATRABAJA', 'COLE_BILINGUE',
       'FAMI_COMECARNEPESCADOHUEVO', 'COLE_NATURALEZA', 'PUNT_GLOBAL'],
      dtype='object')

In [7]:
Genero_dummy = pd.get_dummies(Data[Nombres[2]], prefix = Nombres[1])
Est_padre = pd.get_dummies(Data[Nombres[4]], prefix = Nombres[4])
Est_madre = pd.get_dummies(Data[Nombres[5]], prefix = Nombres[5])
Internet =  pd.get_dummies(Data[Nombres[6]], prefix = Nombres[6])
Compurador = pd.get_dummies(Data[Nombres[7]], prefix = Nombres[7])
Lectura = pd.get_dummies(Data[Nombres[8]], prefix = Nombres[8])
Trabaja = pd.get_dummies(Data[Nombres[9]], prefix = Nombres[9])
Colegio_B =  pd.get_dummies(Data[Nombres[10]], prefix = Nombres[10])
Carne = pd.get_dummies(Data[Nombres[11]], prefix = Nombres[11])
Oficial = pd.get_dummies(Data[Nombres[12]], prefix = Nombres[12])

In [8]:
frames = [Data['EDAD'], Genero_dummy,  Est_padre, Est_madre, Internet, Compurador, Lectura, Trabaja, Colegio_B, Carne, Oficial]

In [9]:
X_r = pd.concat(frames, axis =1) ; X_r

,EDAD,EDAD_F,EDAD_M,FAMI_EDUCACIONPADRE_Educacion profesional completa,FAMI_EDUCACIONPADRE_Educacion profesional incompleta,FAMI_EDUCACIONPADRE_Ninguno,FAMI_EDUCACIONPADRE_No Aplica,FAMI_EDUCACIONPADRE_No sabe,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Tecnica o tecnologica completa,FAMI_EDUCACIONPADRE_Tecnica o tecnologica incompleta,FAMI_EDUCACIONMADRE_Educacion profesional completa,FAMI_EDUCACIONMADRE_Educacion profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Tecnica o tecnologica completa,FAMI_EDUCACIONMADRE_Tecnica o tecnologica incompleta,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si,FAMI_TIENECOMPUTADOR_No,FAMI_TIENECOMPUTADOR_Si,ESTU_DEDICACIONLECTURADIARIA_30 minutos o menos,ESTU_DEDICACIONLECTURADIARIA_Entre 1 y 2 horas,ESTU_DEDICACIONLECTURADIARIA_Entre 30 y 60 minutos,ESTU_DEDICACIONLECTURADIARIA_No leo por entretenimiento,ESTU_DEDICACIONLECTURADIARIA_mas de 2 horas,ESTU_HORASSEMANATRABAJA_0,ESTU_HORASSEMANATRABAJA_Entre 11 y 20 horas,ESTU_HORASSEMANATRABAJA_Entre 21 y 30 horas,ESTU_HORASSEMANATRABAJA_Menos de 10 horas,ESTU_HORASSEMANATRABAJA_mas de 30 horas,COLE_BILINGUE_N,COLE_BILINGUE_S,FAMI_COMECARNEPESCADOHUEVO_1 o 2 veces por semana,FAMI_COMECARNEPESCADOHUEVO_3 a 5 veces por semana,FAMI_COMECARNEPESCADOHUEVO_Nunca o rara vez comemos eso,FAMI_COMECARNEPESCADOHUEVO_Todos o casi todos los dias,COLE_NATURALEZA_NO OFICIAL,COLE_NATURALEZA_OFICIAL
0,18,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0
1,30,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0
2,19,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0
3,11,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0
4,18,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4050,19,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0
4051,25,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0
4052,18,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0
4053,19,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0


In [10]:
Y_r = Data.filter(['PUNT_GLOBAL'])

In [11]:
del X_r['EDAD_F'] , X_r['FAMI_EDUCACIONPADRE_Ninguno'] , X_r['FAMI_EDUCACIONMADRE_Ninguno'], X_r['FAMI_TIENEINTERNET_No'], X_r[ 'FAMI_TIENECOMPUTADOR_No'], X_r['ESTU_DEDICACIONLECTURADIARIA_No leo por entretenimiento'], X_r['ESTU_HORASSEMANATRABAJA_0'], X_r['COLE_BILINGUE_S'], X_r['FAMI_COMECARNEPESCADOHUEVO_Todos o casi todos los dias'] , X_r['COLE_NATURALEZA_NO OFICIAL']

# Regresión Lineal

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X_r, Y_r, test_size = .25, random_state = 20102021)

In [13]:
X_train.shape, X_test.shape, Y_train.shape , Y_test.shape

((3041, 39), (1014, 39), (3041, 1), (1014, 1))

In [14]:
modelo_reg = LinearRegression(fit_intercept= True) 

In [15]:
modelo_reg.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
Y_pred_train = modelo_reg.predict(X_train)
Y_pred_test = modelo_reg.predict(X_test)

In [17]:
def metricas(y_real, y_estimado):
  print(f"Error Cuadratico Medio: {mean_squared_error(y_real, y_estimado)}")
  print(f"Error Absoluto Medio: {mean_absolute_error(y_real, y_estimado)}")
  print(f"Raíz del error cuadrático medio: {np.sqrt(mean_squared_error(y_real, y_estimado))}")

In [18]:
print("Métricas del entrenamiento", end = "\n")
print("---"*10)
metricas(Y_train,Y_pred_train)
print("---"*10)
print("Métricas del Testeo", end = "\n")
print("---"*10)
metricas(Y_test,Y_pred_test)

Métricas del entrenamiento
------------------------------
Error Cuadratico Medio: 1871.2129389481813
Error Absoluto Medio: 34.835338192451566
Raíz del error cuadrático medio: 43.25751887184679
------------------------------
Métricas del Testeo
------------------------------
Error Cuadratico Medio: 1934.6990669387924
Error Absoluto Medio: 35.33593909230676
Raíz del error cuadrático medio: 43.98521418543727


In [19]:
param = {'alpha':[1,10,20,30,50,100,200]}
model = Ridge()
grid = GridSearchCV(model, param, cv = 5)
grid.fit(X_train, Y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1, 10, 20, 30, 50, 100, 200]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [20]:
regresion_final = Ridge(**grid.best_params_)
regresion_final.fit(X_train, Y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [21]:
y_train_fit = regresion_final.predict(X_train)
y_pred = regresion_final.predict(X_test)
print(mean_absolute_error(Y_train, y_train_fit))
print(mean_absolute_error(Y_test, y_pred))

34.84515814178331
35.352145976936235


# Random Forest 

In [22]:
Y = Data.filter(['PUNT_GLOBAL'])

In [23]:
X = Data.drop(['Unnamed: 0','PUNT_GLOBAL','ESTU_DEPTO_RESIDE','FAMI_EDUCACIONPADRE', 'ESTU_DEDICACIONLECTURADIARIA', 'ESTU_HORASSEMANATRABAJA','FAMI_COMECARNEPESCADOHUEVO'], axis = 1); X


,EDAD,ESTU_GENERO,FAMI_EDUCACIONMADRE,FAMI_TIENEINTERNET,FAMI_TIENECOMPUTADOR,COLE_BILINGUE,COLE_NATURALEZA
0,18,F,Educacion profesional completa,Si,Si,S,NO OFICIAL
1,30,F,Secundaria (Bachillerato) completa,Si,Si,N,NO OFICIAL
2,19,M,Secundaria (Bachillerato) incompleta,Si,Si,N,NO OFICIAL
3,11,F,Primaria completa,Si,Si,N,NO OFICIAL
4,18,F,Educacion profesional incompleta,Si,Si,N,NO OFICIAL
...,...,...,...,...,...,...,...
4050,19,M,Educacion profesional completa,Si,Si,N,NO OFICIAL
4051,25,F,Secundaria (Bachillerato) incompleta,Si,Si,N,NO OFICIAL
4052,18,F,Secundaria (Bachillerato) completa,Si,Si,N,NO OFICIAL
4053,19,M,Secundaria (Bachillerato) completa,Si,Si,N,NO OFICIAL


In [24]:
X = pd.get_dummies(X, columns=['ESTU_GENERO'], drop_first= True)
X = pd.get_dummies(X, columns=['FAMI_EDUCACIONMADRE'], drop_first= True)
X = pd.get_dummies(X, columns=['FAMI_TIENEINTERNET'], drop_first= True)
X = pd.get_dummies(X, columns=['FAMI_TIENECOMPUTADOR'], drop_first= True)
X = pd.get_dummies(X, columns=['COLE_BILINGUE'], drop_first= True)
X = pd.get_dummies(X, columns=['COLE_NATURALEZA'], drop_first= True)
#Est_padre = pd.get_dummies(Data[Nombres[4]], prefix = Nombres[4])
#Est_madre = pd.get_dummies(Data[Nombres[5]], prefix = Nombres[5])
#Internet =  pd.get_dummies(Data[Nombres[6]], prefix = Nombres[6])
#Compurador = pd.get_dummies(Data[Nombres[7]], prefix = Nombres[7])
#Lectura = pd.get_dummies(Data[Nombres[8]], prefix = Nombres[8])
#Trabaja = pd.get_dummies(Data[Nombres[9]], prefix = Nombres[9])
#Colegio_B =  pd.get_dummies(Data[Nombres[10]], prefix = Nombres[10])
#Carne = pd.get_dummies(Data[Nombres[11]], prefix = Nombres[11])
#Oficial = pd.get_dummies(Data[Nombres[12]], prefix = Nombres[12])

In [25]:
X

,EDAD,ESTU_GENERO_M,FAMI_EDUCACIONMADRE_Educacion profesional incompleta,FAMI_EDUCACIONMADRE_Ninguno,FAMI_EDUCACIONMADRE_No Aplica,FAMI_EDUCACIONMADRE_No sabe,FAMI_EDUCACIONMADRE_Postgrado,FAMI_EDUCACIONMADRE_Primaria completa,FAMI_EDUCACIONMADRE_Primaria incompleta,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONMADRE_Tecnica o tecnologica completa,FAMI_EDUCACIONMADRE_Tecnica o tecnologica incompleta,FAMI_TIENEINTERNET_Si,FAMI_TIENECOMPUTADOR_Si,COLE_BILINGUE_S,COLE_NATURALEZA_OFICIAL
0,18,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0
1,30,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
2,19,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0
3,11,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0
4,18,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4050,19,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
4051,25,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0
4052,18,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
4053,19,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0


In [26]:
X.columns

Index(['EDAD', 'ESTU_GENERO_M',
       'FAMI_EDUCACIONMADRE_Educacion profesional incompleta',
       'FAMI_EDUCACIONMADRE_Ninguno', 'FAMI_EDUCACIONMADRE_No Aplica',
       'FAMI_EDUCACIONMADRE_No sabe', 'FAMI_EDUCACIONMADRE_Postgrado',
       'FAMI_EDUCACIONMADRE_Primaria completa',
       'FAMI_EDUCACIONMADRE_Primaria incompleta',
       'FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa',
       'FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta',
       'FAMI_EDUCACIONMADRE_Tecnica o tecnologica completa',
       'FAMI_EDUCACIONMADRE_Tecnica o tecnologica incompleta',
       'FAMI_TIENEINTERNET_Si', 'FAMI_TIENECOMPUTADOR_Si', 'COLE_BILINGUE_S',
       'COLE_NATURALEZA_OFICIAL'],
      dtype='object')

In [27]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .25, random_state = 20102021)

In [28]:
X.columns

Index(['EDAD', 'ESTU_GENERO_M',
       'FAMI_EDUCACIONMADRE_Educacion profesional incompleta',
       'FAMI_EDUCACIONMADRE_Ninguno', 'FAMI_EDUCACIONMADRE_No Aplica',
       'FAMI_EDUCACIONMADRE_No sabe', 'FAMI_EDUCACIONMADRE_Postgrado',
       'FAMI_EDUCACIONMADRE_Primaria completa',
       'FAMI_EDUCACIONMADRE_Primaria incompleta',
       'FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) completa',
       'FAMI_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta',
       'FAMI_EDUCACIONMADRE_Tecnica o tecnologica completa',
       'FAMI_EDUCACIONMADRE_Tecnica o tecnologica incompleta',
       'FAMI_TIENEINTERNET_Si', 'FAMI_TIENECOMPUTADOR_Si', 'COLE_BILINGUE_S',
       'COLE_NATURALEZA_OFICIAL'],
      dtype='object')

In [29]:
param = {'n_estimators': [2,4],
         'max_features': [4,6],
         'max_depth': [2,3]}

In [30]:
grid = GridSearchCV(RandomForestRegressor(), param, cv = 5)

In [31]:
mod = RandomForestRegressor()


In [32]:
grid.fit(X_train,Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [33]:
reg_RF = RandomForestRegressor(**grid.best_params_)

In [34]:
reg_RF.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=3, max_features=4, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=4, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [36]:
y_train_fit = reg_RF.predict(X_train)
y_pred = reg_RF.predict(X_test)

In [37]:
metricas(Y_train,y_train_fit)
metricas(Y_test,y_pred)

Error Cuadratico Medio: 2243.3609629389885
Error Absoluto Medio: 38.57177357887698
Raíz del error cuadrático medio: 47.36413160756764
Error Cuadratico Medio: 2262.811394039115
Error Absoluto Medio: 38.43486013173563
Raíz del error cuadrático medio: 47.56901716494797


### xgboost

In [61]:
import xgboost as xgb

In [62]:
X = Data.drop(['Unnamed: 0','PUNT_GLOBAL','ESTU_DEPTO_RESIDE','FAMI_EDUCACIONPADRE', 'ESTU_DEDICACIONLECTURADIARIA', 'ESTU_HORASSEMANATRABAJA','FAMI_COMECARNEPESCADOHUEVO'], axis = 1); X
Y = Data.filter(['PUNT_GLOBAL'])
X

,EDAD,ESTU_GENERO,FAMI_EDUCACIONMADRE,FAMI_TIENEINTERNET,FAMI_TIENECOMPUTADOR,COLE_BILINGUE,COLE_NATURALEZA
0,18,F,Educacion profesional completa,Si,Si,S,NO OFICIAL
1,30,F,Secundaria (Bachillerato) completa,Si,Si,N,NO OFICIAL
2,19,M,Secundaria (Bachillerato) incompleta,Si,Si,N,NO OFICIAL
3,11,F,Primaria completa,Si,Si,N,NO OFICIAL
4,18,F,Educacion profesional incompleta,Si,Si,N,NO OFICIAL
...,...,...,...,...,...,...,...
4050,19,M,Educacion profesional completa,Si,Si,N,NO OFICIAL
4051,25,F,Secundaria (Bachillerato) incompleta,Si,Si,N,NO OFICIAL
4052,18,F,Secundaria (Bachillerato) completa,Si,Si,N,NO OFICIAL
4053,19,M,Secundaria (Bachillerato) completa,Si,Si,N,NO OFICIAL


In [63]:
X = pd.get_dummies(X, columns=['ESTU_GENERO'], drop_first= True)
X = pd.get_dummies(X, columns=['FAMI_EDUCACIONMADRE'], drop_first= True)
X = pd.get_dummies(X, columns=['FAMI_TIENEINTERNET'], drop_first= True)
X = pd.get_dummies(X, columns=['FAMI_TIENECOMPUTADOR'], drop_first= True)
X = pd.get_dummies(X, columns=['COLE_BILINGUE'], drop_first= True)
X = pd.get_dummies(X, columns=['COLE_NATURALEZA'], drop_first= True)

In [75]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.25, random_state=1515)

In [108]:
param1 = {'n_estimators': [4,6,8],
          'objective':['reg:squarederror'],
          'learning_rate':[0.1,0.5,0.8],
          'max_depth':[3,4,5]
          }

In [109]:
grid1 = GridSearchCV(xgb.XGBRegressor(), param1, cv = 5)

In [110]:
grid1.fit(X_train,Y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.5, 0.8],
                         'max_depth': [3, 4, 5], 'n_estimators': [4, 6, 8

In [111]:
reg_XGB = xgb.XGBRegressor(**grid1.best_params_)

In [112]:
reg_XGB.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.5, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=8,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [113]:
y_train_fit = reg_XGB.predict(X_train)
y_pred = reg_XGB.predict(X_test)

In [115]:
print('Métricas del entrenamiento',end='\n')
print('---'*10)
metricas(Y_train, y_train_fit)
print('---'*10)
print('Métricas del testeo')
print('---'*10)
metricas(Y_test, y_pred)

Métricas del entrenamiento
------------------------------
Error Cuadratico Medio: 1751.3452729368219
Error Absoluto Medio: 33.68494424565926
Raíz del error cuadrático medio: 41.8490773247968
------------------------------
Métricas del testeo
------------------------------
Error Cuadratico Medio: 1924.046778943195
Error Absoluto Medio: 35.0440119455552
Raíz del error cuadrático medio: 43.86395762973509
